# .Net Core Data Core Client Demo
This notebook will demonstrate how to use the Intelligent Plant Data Core client in the dotnet interactive Jupyter kernel.

## Installing

Cell magic can be used to install the data core client from the standard nuget package source.

In [ ]:
#r "nuget:IntelligentPlant.DataCore.HttpClient,1.0.0-alpha29"
using IntelligentPlant.DataCore.Client;
using IntelligentPlant.DataCore.Client.Clients;

This cell contains the boiler plate code needed to set up the data core client using the Jupyter server's preconfigured access token.

In [ ]:
using System.Net.Http;
using System.Net.Http.Headers;
using System.Threading;
var accessToken = System.Environment.GetEnvironmentVariable("ACCESS_TOKEN", EnvironmentVariableTarget.Process);
var authenticationHandler = DataCoreHttpClient.CreateAuthenticationMessageHandler(async (HttpRequestMessage request, object context, CancellationToken ct) => {
    return new AuthenticationHeaderValue("Bearer", accessToken);
});
var primaryHandler = new SocketsHttpHandler();
authenticationHandler.InnerHandler = primaryHandler;
var baseClient = new HttpClient(authenticationHandler);
var client = new DataCoreHttpClient<object, DataCoreHttpClientOptions>(baseClient, new DataCoreHttpClientOptions() {
    DataCoreUrl = new System.Uri("https://appstore.intelligentplant.com/gestalt/datacore/")
});

The data core client is now configured.

The client can be used to get a list of available data sources (that have been authorised for the Jupyter Hub app).

In [ ]:
var datasourceResp = await client.DataSources.GetDataSourcesAsync();

An expression without a ';' will be displayed as the cell output.

In [ ]:
datasourceResp.Select(x => x.Name.QualifiedName)

The client can be used to query for available tags.

In this case we'll get the 1st 5 tags from the demo data source "IP Data source 2".

(Make sure this data source appears in the list of authorised data sources)

In [ ]:
var dsn = "IP Datasource 2";

In [ ]:
var resp = await client.DataSources.FindTagsAsync(dsn, nameFilter: "*", page: 1, pageSize: 5);

In [ ]:
var tags = resp.Select(x => x.Name).ToArray();

In [ ]:
tags

To query a data source for tag data a tag map must be created.

The map should have qualified data source names as keys and arrays of tag names as values.

This tag map is setup to query "IP Datasource 2" for the 1st 5 tags that we requested from it earlier.

In [ ]:
var tagMap = new Dictionary<string, string[]>();
tagMap[dsn] = tags;

To query for plot data use the `ReadPlotTagValuesAsync(..)` function. This function takes a tag map, query start time, query end time and number of samples as parameters.

In [ ]:
var dataResp = await client.DataSources.ReadPlotTagValuesAsync(tagMap, "*-30d", "*", 100);

The response is a dictionary where the keys are the data source names are the keys.

The value for each data source is a nested dictionary where the keys are the requested tag names and the values are a historical tag value object, which contains an array of tag values.

In [ ]:
dataResp.Keys

In [ ]:
dataResp[dsn].Keys

In [ ]:
dataResp[dsn][tags.First()]

The each tag value is an object containg meta data and the tag value.

Depending on the data function the sample times may not be at consistent intervals and therefore may differ from 1 tag to another.

In [ ]:
dataResp[dsn][tags.First()].Values.First()

To extract the numeric data for a given tag use Linq

In [ ]:
dataResp[dsn][tags.First()].Values.Select(val => val.NumericValue)

The XPlot Library lets us easily plot the tag data.

In [ ]:
using XPlot.Plotly;

In [ ]:
var tagSeries = new List<Graph.Scatter>();

foreach (var tag in tags) {
    
    var data = dataResp[dsn][tag].Values.Select(x => x.NumericValue);
    
    var min = data.Min();
    
    var max = data.Max() - min;
    
    var series = new Graph.Scatter
    {
        name = tag,
        x = dataResp[dsn][tag].Values.Select(x => x.UtcSampleTime),
        y = data.Select(x => (x - min)/max) //normalise the data into the range 0-1
    };
    
    tagSeries.Add(series);
}

var chart = Chart.Plot(tagSeries);
display(chart);

In [ ]:
var openSeries = new Graph.Scatter
{
    mode = "markers",
    x = dataResp[dsn]["Anti-Scalent Pump A_PI"].Values.Select(x => x.NumericValue),
    y = dataResp[dsn]["Anti-Scalent Pump B_PI"].Values.Select(x => x.NumericValue)
};


var chart = Chart.Plot(new[] {openSeries});
chart.WithTitle("Anti-Scalent Pump A_PI vs Anti-Scalent Pump B_PI");
display(chart);